In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("/content/drive/MyDrive/Hackaholic/Data/India Agriculture Crop Production Dataset/India Agriculture Crop Production.csv")
data

,State,District,Crop,Year,Season,Area,Area Units,Production,Production Units,Yield
0,Andaman and Nicobar Islands,NICOBARS,Arecanut,2001-02,Kharif,1254.0,Hectare,2061.0,Tonnes,1.643541
1,Andaman and Nicobar Islands,NICOBARS,Arecanut,2002-03,Whole Year,1258.0,Hectare,2083.0,Tonnes,1.655803
2,Andaman and Nicobar Islands,NICOBARS,Arecanut,2003-04,Whole Year,1261.0,Hectare,1525.0,Tonnes,1.209358
3,Andaman and Nicobar Islands,NORTH AND MIDDLE ANDAMAN,Arecanut,2001-02,Kharif,3100.0,Hectare,5239.0,Tonnes,1.690000
4,Andaman and Nicobar Islands,SOUTH ANDAMANS,Arecanut,2002-03,Whole Year,3105.0,Hectare,5267.0,Tonnes,1.696296
...,...,...,...,...,...,...,...,...,...,...
345402,Manipur,IMPHAL WEST,NaN,2019-20,Rabi,NaN,Hectare,NaN,Tonnes,NaN
345403,Manipur,SENAPATI,NaN,2019-20,Rabi,NaN,Hectare,NaN,Tonnes,NaN
345404,Manipur,TAMENGLONG,NaN,2019-20,Rabi,NaN,Hectare,NaN,Tonnes,NaN
345405,Manipur,THOUBAL,NaN,2019-20,Rabi,NaN,Hectare,NaN,Tonnes,NaN


In [3]:
import pickle
with open('/content/drive/MyDrive/Hackaholic/models.sav','rb') as reg_file:
    models=pickle.load(reg_file)
models

{'Andaman and Nicobar Islands': LinearRegression(),
 'Andhra Pradesh': LinearRegression(),
 'Arunachal Pradesh': LinearRegression(),
 'Assam': LinearRegression(),
 'Bihar': LinearRegression(),
 'Chandigarh': LinearRegression(),
 'Chhattisgarh': LinearRegression(),
 'Dadra and Nagar Haveli': LinearRegression(),
 'Daman and Diu': LinearRegression(),
 'Delhi': LinearRegression(),
 'Goa': LinearRegression(),
 'Gujarat': LinearRegression(),
 'Haryana': LinearRegression(),
 'Himachal Pradesh': LinearRegression(),
 'Jammu and Kashmir': LinearRegression(),
 'Jharkhand': LinearRegression(),
 'Karnataka': LinearRegression(),
 'Kerala': LinearRegression(),
 'Madhya Pradesh': LinearRegression(),
 'Maharashtra': LinearRegression(),
 'Manipur': LinearRegression(),
 'Meghalaya': LinearRegression(),
 'Mizoram': LinearRegression(),
 'Nagaland': LinearRegression(),
 'Odisha': LinearRegression(),
 'Puducherry': LinearRegression(),
 'Punjab': LinearRegression(),
 'Rajasthan': LinearRegression(),
 'Sikkim'

In [4]:
# data=data.drop(['District',"Year"],axis=1)

le_season=LabelEncoder()
le_season.fit(data['Season'])
data['Season']=le_season.transform(data['Season'])

le_pu=LabelEncoder()
le_pu.fit(data['Production Units'])
data['Production Units']=le_pu.transform(data['Production Units'])

# area_unit=data['Area Units'].unique()[0]
# data=data.drop(['Area Units'],axis=1)

# crops=data['Crop'].unique()
# states=data['State'].unique()

In [5]:
# st="Andaman and Nicobar Islands"
data=data.dropna()
data=pd.get_dummies(data,columns=['Crop'])

In [6]:
class DemandPredictor:
    '''This is a Demand Forecasting model, predicting crop yield values'''
    def __init__(self,models):
        self.area=0
        self.state=''
        self.crop=''
        self.season=''
        self.models=models

    def inputData(self):
        self.state=input("Enter your State:").strip()
        self.crop=input("Enter your crop:").strip()
        self.area=float(input("Enter the area of cultivation (in hectares):"))
        self.season=input("Enter the season:")

    def setup(self):
        self.curr_model=self.models[self.state]
        self.iparr=np.zeros(len(self.curr_model.feature_names_in_))
        self.iparr[0]=le_season.transform([self.season])
        self.iparr[1]=self.area
        self.par='Crop_'+self.crop
        ind=np.where(self.curr_model.feature_names_in_==self.par)[0][0]
        self.iparr[ind]=1
        # print(self.iparr)

    def predict(self):
        self.pred_yield=self.curr_model.predict([self.iparr])
        # print(self.pred_yield)

    def fetch_current_yield(self):
        self.curr_yield=65.000
    
    def demand_compare(self):
        self.los=5
        if(self.area>1000):
            self.los=1
        self.pred_area=(self.pred_yield/self.curr_yield)*self.area
        self.yield_diff=100*(self.pred_yield-self.curr_yield)/self.curr_yield
        op="{0}'s demand remains the same\n You can continue with the current cultivation area".format(self.crop)
        if(self.pred_yield>self.curr_yield):
            if(self.yield_diff > self.los):
                op="{0}'s demand is High in the Future!!!\nTo gain more profit, increase cultivation area by {1} hectares".format(self.crop,(self.pred_area-self.area)[0])
        elif(self.pred_yield<self.curr_yield):
            if(self.yield_diff < -self.los):
                op="{0}'s demand is Low in the Future!!!\n To prevent losses, decrease cultivation area by {1} hectares".format(self.crop,(self.area-self.pred_area)[0])
        print(op)

    def run(self):
        self.inputData()
        self.setup()
        self.predict()
        self.fetch_current_yield()
        self.demand_compare()

In [7]:
sm=models['Tamil Nadu']
help(sm)

Help on LinearRegression in module sklearn.linear_model._base object:

class LinearRegression(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, LinearModel)
 |  LinearRegression(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
 |  
 |  Ordinary least squares Linear Regression.
 |  
 |  LinearRegression fits a linear model with coefficients w = (w1, ..., wp)
 |  to minimize the residual sum of squares between the observed targets in
 |  the dataset, and the targets predicted by the linear approximation.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : bool, default=True
 |      Whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (i.e. data is expected to be centered).
 |  
 |  copy_X : bool, default=True
 |      If True, X will be copied; else, it may be overwritten.
 |  
 |  n_jobs : int, default=None
 |      The number of jobs to use for the computation. This will only provide
 | 

In [8]:
sm.feature_names_in_

array(['Season', 'Area', 'Crop_Arhar/Tur', 'Crop_Bajra', 'Crop_Banana',
       'Crop_Cashewnut', 'Crop_Castor seed', 'Crop_Coconut',
       'Crop_Coriander', 'Crop_Cotton(lint)', 'Crop_Cowpea(Lobia)',
       'Crop_Dry chillies', 'Crop_Garlic', 'Crop_Ginger', 'Crop_Gram',
       'Crop_Groundnut', 'Crop_Horse-gram', 'Crop_Jowar', 'Crop_Linseed',
       'Crop_Maize', 'Crop_Masoor', 'Crop_Mesta',
       'Crop_Moong(Green Gram)', 'Crop_Niger seed', 'Crop_Oilseeds total',
       'Crop_Onion', 'Crop_Other Kharif pulses', 'Crop_Other Rabi pulses',
       'Crop_Potato', 'Crop_Ragi', 'Crop_Rapeseed &Mustard', 'Crop_Rice',
       'Crop_Safflower', 'Crop_Sesamum', 'Crop_Small millets',
       'Crop_Soyabean', 'Crop_Sugarcane', 'Crop_Sunflower',
       'Crop_Sweet potato', 'Crop_Tobacco', 'Crop_Turmeric', 'Crop_Urad',
       'Crop_Wheat', 'Crop_other oilseeds'], dtype=object)

In [9]:
help(le_season)
le_season.transform(['Kharif'])

Help on LabelEncoder in module sklearn.preprocessing._label object:

class LabelEncoder(sklearn.base.TransformerMixin, sklearn.base.BaseEstimator)
 |  Encode target labels with value between 0 and n_classes-1.
 |  
 |  This transformer should be used to encode target values, *i.e.* `y`, and
 |  not the input `X`.
 |  
 |  Read more in the :ref:`User Guide <preprocessing_targets>`.
 |  
 |  .. versionadded:: 0.12
 |  
 |  Attributes
 |  ----------
 |  classes_ : ndarray of shape (n_classes,)
 |      Holds the label for each class.
 |  
 |  See Also
 |  --------
 |  OrdinalEncoder : Encode categorical features using an ordinal encoding
 |      scheme.
 |  OneHotEncoder : Encode categorical features as a one-hot numeric array.
 |  
 |  Examples
 |  --------
 |  `LabelEncoder` can be used to normalize labels.
 |  
 |  >>> from sklearn import preprocessing
 |  >>> le = preprocessing.LabelEncoder()
 |  >>> le.fit([1, 2, 2, 6])
 |  LabelEncoder()
 |  >>> le.classes_
 |  array([1, 2, 6])
 |  >

array([1])

In [12]:
d_pred=DemandPredictor(models)
d_pred.run()

Enter your State:Tamil Nadu
Enter your crop:Banana
Enter the area of cultivation (in hectares):0.2
Enter the season:Kharif
Banana's demand is Low in the Future!!!
 To prevent losses, decrease cultivation area by 0.030876718006801862 hectares


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
